# Import Lib

In [95]:
from keras.models import Sequential
from keras import layers
import numpy as np
from keras.models import Model
from keras.layers import *
from six.moves import range

# Parameters Config

In [8]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [9]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [10]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [11]:
ctable = CharacterTable(chars)

In [12]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [13]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [14]:
print(questions[:5], expected[:5])

['780+162', '299+1  ', '1+888  ', '59+8   ', '71+652 '] ['942 ', '300 ', '889 ', '67  ', '723 ']


# Processing

In [15]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [16]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [17]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False  True False
   False]
  [False False False  True False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False  True False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False False False  True False False Fal

# Build Model

In [18]:
x_train.shape[1:]

(7, 12)

In [99]:
print('Build model...')

##### Build your own model here ############
# input to LSTM should have shape (nb_of_samples, seq_len, features)
# 
def my_model(input_data):
    
    model = Sequential()
    model.add(layers.LSTM(HIDDEN_SIZE, input_shape=(input_data[1], input_data[2]), name="lstm_1") )
    model.add(layers.RepeatVector(4, name="repeat_vector"))
    model.add(layers.LSTM(HIDDEN_SIZE, return_sequences=True, name="lstm_2"))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax'), name="time_distributed"))

    model.summary()
    
    return model

Build model...


In [100]:
x_train.shape[1]

7

In [103]:
model = my_model(x_train.shape)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [104]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 268us/step - loss: 2.0245 - acc: 0.2881 - val_loss: 1.8725 - val_acc: 0.3240
Q 916+746 T 1662 ☒ 110 
Q 165+8   T 173  ☒ 11  
Q 880+90  T 970  ☒ 110 
Q 16+362  T 378  ☒ 119 
Q 884+62  T 946  ☒ 110 
Q 776+656 T 1432 ☒ 110 
Q 719+6   T 725  ☒ 119 
Q 178+530 T 708  ☒ 110 
Q 475+56  T 531  ☒ 110 
Q 13+35   T 48   ☒ 16  

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 159us/step - loss: 1.8586 - acc: 0.3281 - val_loss: 1.8383 - val_acc: 0.3327
Q 175+73  T 248  ☒ 108 
Q 58+750  T 808  ☒ 108 
Q 906+813 T 1719 ☒ 1081
Q 308+696 T 1004 ☒ 100 
Q 652+96  T 748  ☒ 108 
Q 78+167  T 245  ☒ 108 
Q 8+950   T 958  ☒ 108 
Q 481+70  T 551  ☒ 108 
Q 78+833  T 911  ☒ 108 
Q 46+432  T 478  ☒ 104 

-------------------

18000/18000 [==============================] - 3s 161us/step - loss: 1.2239 - acc: 0.5468 - val_loss: 1.2206 - val_acc: 0.5461
Q 89+26   T 115  ☒ 119 
Q 138+7   T 145  ☒ 181 
Q 342+918 T 1260 ☒ 1235
Q 175+709 T 884  ☒ 962 
Q 320+77  T 397  ☒ 302 
Q 86+488  T 574  ☒ 533 
Q 385+438 T 823  ☒ 873 
Q 843+542 T 1385 ☒ 1488
Q 808+654 T 1462 ☒ 1553
Q 15+130  T 145  ☒ 155 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 164us/step - loss: 1.1838 - acc: 0.5607 - val_loss: 1.1955 - val_acc: 0.5536
Q 301+8   T 309  ☒ 311 
Q 407+323 T 730  ☒ 744 
Q 40+313  T 353  ☒ 445 
Q 948+6   T 954  ☒ 960 
Q 31+234  T 265  ☒ 346 
Q 895+81  T 976  ☒ 900 
Q 122+92  T 214  ☒ 295 
Q 272+64  T 336  ☒ 301 
Q 66+20   T 86   ☒ 68  
Q 288+47  T 335  ☒ 338 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 175us/step - loss: 0.1686 - acc: 0.9666 - val_loss: 0.2058 - val_acc: 0.9429
Q 4+704   T 708  ☑ 708 
Q 328+7   T 335  ☑ 335 
Q 510+528 T 1038 ☑ 1038
Q 96+895  T 991  ☑ 991 
Q 223+2   T 225  ☑ 225 
Q 212+222 T 434  ☑ 434 
Q 878+67  T 945  ☑ 945 
Q 743+2   T 745  ☑ 745 
Q 842+836 T 1678 ☑ 1678
Q 31+647  T 678  ☑ 678 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 162us/step - loss: 0.1495 - acc: 0.9734 - val_loss: 0.1846 - val_acc: 0.9492
Q 192+13  T 205  ☑ 205 
Q 196+937 T 1133 ☒ 1123
Q 62+350  T 412  ☑ 412 
Q 668+382 T 1050 ☑ 1050
Q 4+151   T 155  ☑ 155 
Q 287+575 T 862  ☑ 862 
Q 334+75  T 409  ☑ 409 
Q 100+84  T 184  ☑ 184 
Q 337+79  T 416  ☑ 416 
Q 935+14  T 949  ☑ 949 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 164us/step - loss: 0.0680 - acc: 0.9806 - val_loss: 0.0819 - val_acc: 0.9740
Q 73+319  T 392  ☑ 392 
Q 705+364 T 1069 ☑ 1069
Q 111+76  T 187  ☑ 187 
Q 212+222 T 434  ☑ 434 
Q 419+23  T 442  ☑ 442 
Q 692+957 T 1649 ☑ 1649
Q 918+148 T 1066 ☑ 1066
Q 83+476  T 559  ☑ 559 
Q 251+95  T 346  ☑ 346 
Q 56+994  T 1050 ☑ 1050

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 161us/step - loss: 0.0266 - acc: 0.9970 - val_loss: 0.0688 - val_acc: 0.9800
Q 152+902 T 1054 ☑ 1054
Q 988+84  T 1072 ☑ 1072
Q 480+519 T 999  ☑ 999 
Q 6+971   T 977  ☑ 977 
Q 232+600 T 832  ☑ 832 
Q 886+624 T 1510 ☑ 1510
Q 545+4   T 549  ☑ 549 
Q 15+493  T 508  ☑ 508 
Q 366+79  T 445  ☑ 445 
Q 921+998 T 1919 ☑ 1919

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 161us/step - loss: 0.0149 - acc: 0.9981 - val_loss: 0.0511 - val_acc: 0.9831
Q 886+4   T 890  ☑ 890 
Q 2+287   T 289  ☑ 289 
Q 0+433   T 433  ☑ 433 
Q 148+950 T 1098 ☑ 1098
Q 16+362  T 378  ☑ 378 
Q 328+86  T 414  ☑ 414 
Q 832+10  T 842  ☑ 842 
Q 886+624 T 1510 ☑ 1510
Q 5+404   T 409  ☑ 409 
Q 51+58   T 109  ☒ 119 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 168us/step - loss: 0.0079 - acc: 0.9998 - val_loss: 0.0484 - val_acc: 0.9841
Q 4+444   T 448  ☑ 448 
Q 486+3   T 489  ☑ 489 
Q 95+256  T 351  ☑ 351 
Q 64+481  T 545  ☑ 545 
Q 657+56  T 713  ☑ 713 
Q 590+792 T 1382 ☑ 1382
Q 349+687 T 1036 ☑ 1036
Q 19+903  T 922  ☑ 922 
Q 5+908   T 913  ☑ 913 
Q 567+27  T 594  ☒ 694 


# Testing

In [109]:
def vectorization(ary_x, ary_y):
    print('Vectorization...')
    x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(test_x):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(test_y):
        y[i] = ctable.encode(sentence, DIGITS + 1)
    return x, y

In [115]:
def evaluation_test(x_test, y_test, model):
    x_val, y_val = vectorization(test_x, test_y)
    for i in range(len(y_val)):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

In [116]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
# test_x = ["555+175", "860+7  ", "340+29 "]
# test_y = ["730 ", "867 ", "369 "] 

# evaluation_test(test_x, test_y, model)
#####################################################
testing_callback = model.evaluate(x=test_x, y=test_y, batch_size=1, verbose=1)
for i in range(len(model.metrics_names)):
        print(model.metrics_names[i]+" : "+str(testing_callback[i]))

MSG : Prediction
Vectorization...
Q 340+29  T 369  ☑ 369 
Q 555+175 T 730  ☑ 730 
Q 555+175 T 730  ☑ 730 
